In [1]:
! pip install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

import FinanceDataReader as fdr
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
random.seed(1234)
np.random.seed(1234)
os.environ["PYTHONHASHSEED"] = str(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = '/content/drive/MyDrive/주식 종가 예측/open/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [6]:
start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 47
HOW MANY "Business_days" : (235, 1)


# Sample

In [42]:
sample_code = stock_list.loc[100, '종목코드']
sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample.fillna(method='ffill', inplace=True)

In [43]:
import warnings
warnings.filterwarnings("ignore")

# 두 기간에 대한 예측 시행
train = sample[:-20] 

p = range(0, 5)
d = range(0, 2)
q = range(0, 5)
pdq = list(itertools.product(p, d, q))

gridsearch_result = pd.DataFrame(columns = ['pdq', 'trend', 'aic'])

for pdq_param in pdq:
  try:
    model = ARIMA(train['Close'].values.astype('float64'), order = pdq_param)
    model_fitted = model.fit(trend = 'c', full_output = True, disp = True)
          
    aic = model_fitted.aic

    gridsearch_result = gridsearch_result.append({'pdq' : pdq_param, 'trend' : trend_param, 'aic' : aic}, ignore_index = True)
        
  except:
    continue

In [45]:
min_idx = gridsearch_result[['aic']].idxmin()

optimal_pdq = gridsearch_result.loc[min_idx, 'pdq'].item()
optimal_trend = gridsearch_result.loc[min_idx, 'trend'].item()

# Final train / test set
train_public = sample[:-20]
test_public = sample[-20:-15]['Close'].reset_index(drop = True)
test_public = np.array(test_public)

# Fit the model with selected p, d, q values
optimal_model = ARIMA(train_public['Close'].values.astype('float64'), order = optimal_pdq)
optimal_model_fitted = optimal_model.fit(trend = optimal_trend, full_output = True, disp = True)
optimal_forecast_result = optimal_model_fitted.forecast(steps = 5)

pred_public = optimal_forecast_result[0]

# Check nmae values
public_nmae = np.mean(np.abs(test_public - pred_public) / test_public)
  
print('predicted values: ', pred_public, '\n')
print('nmae: ', public_nmae, '\n')

predicted values:  [105173.78753102 105327.67039492 105447.83059623 105537.64379037
 105600.24080833] 

nmae:  0.006119119874643089 



# Final

In [7]:
nmae_df = pd.DataFrame(columns = ['code', 'nmae'])

In [8]:
submission_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,submission_name))

In [ ]:
import warnings
import time

warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
  print(f'\n code: {code} \n')

  # Load data
  stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_data.fillna(method = 'ffill', inplace = True)

  # Split the data into train / validation / test sets
  train_public = stock_data[:-20] # ~ 10/29 : predict from 11/1 to 11/5
  train_private = stock_data # ~ 11/26 : predict from 11/29 to 12/3

  test_public = stock_data[-20:-15]['Close'].reset_index(drop = True) # from 11/1 to 11/5
  test_public = np.array(test_public)

  p = range(0, 5)
  d = range(0, 2)
  q = range(0, 5)
  trend = ('n', 'c', 't', 'ct')
  pdq = list(itertools.product(p, d, q))

  public_gridsearch_result = pd.DataFrame(columns = ['pdq', 'trend', 'aic'])

  for pdq_param in pdq:
    for trend_param in trend:
      try:
        model = ARIMA(train_public['Close'].values.astype('float64'), order = pdq_param)
        model_fitted = model.fit(trend = trend_param, full_output = True, disp = True)
              
        aic = model_fitted.aic

        public_gridsearch_result = public_gridsearch_result.append({'pdq' : pdq_param, 'trend' : trend_param, 'aic' : aic}, ignore_index = True)
            
      except:
        continue
  
  min_idx = public_gridsearch_result[['aic']].idxmin()

  optimal_pdq = public_gridsearch_result.loc[min_idx, 'pdq'].item()
  optimal_trend = public_gridsearch_result.loc[min_idx, 'trend'].item()

  # Fit the model with selected p, d, q values
  public_model = ARIMA(train_public['Close'].values.astype('float64'), order = optimal_pdq)
  public_model_fitted = public_model.fit(trend = optimal_trend, full_output = True, disp = True)
  public_forecast_result = public_model_fitted.forecast(steps = 5)

  pred_public = public_forecast_result[0]

  # Check nmae values
  public_nmae = np.mean(np.abs(test_public - pred_public) / test_public)
  nmae_df = nmae_df.append({'code' : code, 'nmae' : public_nmae}, ignore_index = True)
    
  print('public predicted values: ', pred_public, '\n')
  print('nmae: ', public_nmae, '\n')

  # Add predicted valued to submission file
  for i in range (0, 5):
    submission.loc[i, code] = pred_public[i]

  private_gridsearch_result = pd.DataFrame(columns = ['pdq', 'trend', 'aic'])

  for pdq_param in pdq:
    for trend_param in trend:
      try:
        model = ARIMA(train_private['Close'].values.astype('float64'), order = pdq_param)
        model_fitted = model.fit(trend = trend_param, full_output = True, disp = True)
              
        aic = model_fitted.aic

        private_gridsearch_result = private_gridsearch_result.append({'pdq' : pdq_param, 'trend' : trend_param, 'aic' : aic}, ignore_index = True)
            
      except:
        continue

  min_idx = private_gridsearch_result[['aic']].idxmin()

  optimal_pdq = private_gridsearch_result.loc[min_idx, 'pdq'].item()
  optimal_trend = private_gridsearch_result.loc[min_idx, 'trend'].item()

  # Fit the model with selected p, d, q values
  private_model = ARIMA(train_private['Close'].values.astype('float64'), order = optimal_pdq)
  private_model_fitted = private_model.fit(trend = optimal_trend, full_output = True, disp = True)
  private_forecast_result = private_model_fitted.forecast(steps = 5)

  pred_private = private_forecast_result[0]

  print('private predicted values: ', pred_private, '\n')

  # Add predicted valued to submission file
  for i in range (0, 5):
    submission.loc[i+5, code] = pred_private[i]

  time.sleep(1)

  0%|          | 0/370 [00:00<?, ?it/s]


 code: 005930 

public predicted values:  [69828.77108485 70283.82160995 70681.87706018 70700.09423778
 70882.09300165] 

nmae:  0.006633325016175867 

private predicted values:  [71942.26411768 71818.70351338 71930.01716003 71749.72292706
 71586.35738039] 



  0%|          | 1/370 [00:34<3:32:19, 34.52s/it]


 code: 000660 

public predicted values:  [103362.30252274 103181.28306306 103076.69437876 102961.35394206
 102847.52598746] 

nmae:  0.032016084259835716 

private predicted values:  [115702.9450933  115621.74518497 115577.19547831 115527.91847652
 115479.25122087] 



  1%|          | 2/370 [01:02<3:07:22, 30.55s/it]

...

 99%|█████████▉| 368/370 [3:12:21<01:02, 31.29s/it]


 code: 110790 

public predicted values:  [42796.46229153 42851.44769856 43053.01648373 43266.21073225
 43510.73324738] 

nmae:  0.030012635475585758 

private predicted values:  [46918.52832916 46657.17084162 46711.96416714 46515.7739893
 46512.21007574] 



100%|█████████▉| 369/370 [3:12:51<00:31, 31.03s/it]


 code: 099320 

public predicted values:  [50289.90147783 50379.80295567 50469.7044335  50559.60591133
 50649.50738916] 

nmae:  0.04434017779986474 

private predicted values:  [45410.0896861  45470.1793722  45530.2690583  45590.35874439
 45650.44843049] 



100%|██████████| 370/370 [3:13:24<00:00, 31.36s/it]


In [ ]:
# nmae 평균
nmae_result = np.array(nmae_df['nmae'])

In [ ]:
np.mean(nmae_result)

0.017420893394797365